## Setup

In [ ]:
# from google.colab import auth
# from google.cloud import bigquery
# from google.colab import data_table
# from google.colab import widgets

# from collections import Counter
# import re
# import numpy as np
# import pandas as pd
# import math



In [ ]:
# import os
# from google.colab import drive
# import sys

# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/HML/Final Project')
# sys.path.append(os.path.abspath('/content/drive/MyDrive/HML/Final Project'))

In [ ]:
# !pip install duckdb --quiet
# import duckdb

In [ ]:
# drive_path = '/content/drive/MyDrive/HML/Final Project/MIMIC-III'
# con = duckdb.connect(f'{drive_path}/mimiciii.duckdb')

In [5]:
import pandas as pd
import numpy as np
from tqdm import tqdm 

import torch
from torch_geometric.nn import GATv2Conv
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-08-21 09:41:32.336309: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755758492.349580 3463189 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755758492.353730 3463189 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755758492.364581 3463189 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755758492.364593 3463189 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755758492.364595 3463189 computation_placer.cc:177] computation placer alr

## Loading data

In [ ]:
subject_ids = pd.read_csv('data/initial_cohort.csv')['subject_id'].to_list()
lavbevent_meatdata = pd.read_csv('data/labs_metadata.csv')
vital_meatdata = pd.read_csv('data/vital_metadata.csv')

In [7]:
ICUQ = \
"""--sql
SELECT admissions.subject_id::INTEGER AS subject_id, admissions.hadm_id::INTEGER AS hadm_id
, admissions.admittime::DATE AS admittime, admissions.dischtime::DATE AS dischtime
, admissions.ethnicity, admissions.deathtime::DATE AS deathtime
, patients.gender, patients.dob::DATE AS dob, icustays.icustay_id::INTEGER AS icustay_id, patients.dod::DATE as dod,
icustays.intime::DATE AS intime,icustays.outtime::DATE AS outtime
FROM admissions
INNER JOIN patients
    ON admissions.subject_id = patients.subject_id
LEFT JOIN icustays
    ON admissions.hadm_id = icustays.hadm_id

WHERE admissions.has_chartevents_data = 1
AND admissions.subject_id::INTEGER IN ?
ORDER BY admissions.subject_id, admissions.hadm_id, admissions.admittime;
"""

# icu =  con.execute(ICUQ, [subject_ids]).fetchdf().rename(str.lower, axis='columns')


In [8]:
LABQUERY = \
f"""--sql
SELECT labevents.subject_id::INTEGER AS subject_id\
      , labevents.hadm_id::INTEGER AS hadm_id\
      , labevents.charttime::DATE AS charttime
      , labevents.itemid::INTEGER AS itemid\
      , labevents.valuenum::DOUBLE AS valuenum
      , admissions.admittime::DATE AS admittime
FROM labevents
          INNER JOIN admissions
                    ON labevents.subject_id = admissions.subject_id
                        AND labevents.hadm_id = admissions.hadm_id
                        AND labevents.charttime::DATE between
                            (admissions.admittime::DATE)
                            AND (admissions.admittime::DATE + interval 48 hour)
                        AND itemid::INTEGER IN ? \
                        """

VITQUERY = f"""--sql
        SELECT chartevents.subject_id::INTEGER AS subject_id\
             , chartevents.hadm_id::INTEGER AS hadm_id\
             , chartevents.charttime::DATE AS charttime\
             , chartevents.itemid::INTEGER AS itemid\
             , chartevents.valuenum::DOUBLE AS valuenum\
             , admissions.admittime::DATE AS admittime\
        FROM chartevents
                 INNER JOIN admissions
                            ON chartevents.subject_id = admissions.subject_id
                                AND chartevents.hadm_id = admissions.hadm_id
                                AND chartevents.charttime::DATE between
                                   (admissions.admittime::DATE)
                                   AND (admissions.admittime::DATE + interval 48 hour)
                                AND itemid::INTEGER in ?
      -- exclude rows marked as error
      AND chartevents.error::INTEGER IS DISTINCT \
        FROM 1 \
        """


# lab = con.execute(LABQUERY, [lavbevent_meatdata['itemid'].tolist()]).fetchdf().rename(str.lower, axis='columns')
# vit = con.execute(VITQUERY, [vital_meatdata['itemid'].tolist()]).fetchdf().rename(str.lower, axis='columns')

In [9]:
pred_window = 13*24      # duration of prediction window (hours)
pred_gap = 24            # minimal gap between prediction and target (hours)
min_los = 24             # minimal length of stay (hours)
min_target_onset = 2*24  # minimal time of target since admission (hours)
pred_freq = '4H'        # prediction frequency

labs = pd.read_csv('data/labs.csv')
vits = pd.read_csv('data/vits.csv')
hosps = pd.read_csv('data/icu.csv')

for col in ['admittime', 'dischtime', 'dob', 'dod', 'intime', 'outtime']:
    hosps[col] = pd.to_datetime(hosps[col].str.strip(), errors='coerce')

In [10]:
bios = pd.read_csv('data/bios.csv')
bios['charttime'] = pd.to_datetime(bios['charttime'].str.strip(), errors='coerce')

## preprocessing

In [11]:
# ethnicity  - to category
hosps.ethnicity = hosps.ethnicity.str.lower()
hosps.loc[(hosps.ethnicity.str.contains('^white')),'ethnicity'] = 'white'
hosps.loc[(hosps.ethnicity.str.contains('^black')),'ethnicity'] = 'black'
hosps.loc[(hosps.ethnicity.str.contains('^hisp')) | (hosps.ethnicity.str.contains('^latin')),'ethnicity'] = 'hispanic'
hosps.loc[(hosps.ethnicity.str.contains('^asia')),'ethnicity'] = 'asian'
hosps.loc[~(hosps.ethnicity.str.contains('|'.join(['white', 'black', 'hispanic', 'asian']))),'ethnicity'] = 'other'

# ethnicity - one hot encoding
hosps['eth_white'] = (hosps['ethnicity'] == 'white').astype(int)
hosps['eth_black'] = (hosps['ethnicity'] == 'black').astype(int)
hosps['eth_hispanic'] = (hosps['ethnicity'] == 'hispanic').astype(int)
hosps['eth_asian'] = (hosps['ethnicity'] == 'asian').astype(int)
hosps['eth_other'] = (hosps['ethnicity'] == 'other').astype(int)
hosps.drop(['ethnicity', 'deathtime'], inplace=True, axis=1)

In [22]:
bios_onehot = pd.get_dummies(bios, columns=['org_itemid'], prefix='org')
groupby_cols = ['subject_id', 'hadm_id', 'charttime']
onehot_cols = [col for col in bios_onehot.columns if col.startswith('org_')]
bios_table = bios_onehot.groupby(groupby_cols)[onehot_cols].sum().reset_index()
for col in onehot_cols:
    bios_table[col] = (bios_table[col] > 0).astype(int)

In [ ]:

# Generate feature columns for los, age and mortality
def age(admittime, dob):
    if admittime < dob:
      return 0
    return admittime.year - dob.year - ((admittime.month, admittime.day) < (dob.month, dob.day))

hosps['age'] = hosps.apply(lambda row: age(row['admittime'], row['dob']), axis=1)
hosps['los_hosp_hr'] = (hosps.dischtime - hosps.admittime).dt.total_seconds()/3600
hosps['mort'] = np.where(~np.isnat(hosps.dod),1,0)

# Gender to binary
hosps['gender'] = np.where(hosps['gender'] == "M", 1, 0)

# @title Q1.1 - Patient Exclusion Criteria
hosps = hosps.sort_values('admittime').groupby('subject_id').first().reset_index()
print(f"1. Include only first admissions: N={hosps.shape[0]}")

hosps = hosps[hosps.age.between(18,90)]
print(f"2. Exclusion by ages: N={hosps.shape[0]}")

# Exclude patients hospitalized for less than 24 hours
hosps = hosps[hosps['los_hosp_hr'] >= min_los]
print(f"3. Include only patients who admitted for at least {min_los} hours: N={hosps.shape[0]}")

# Exclude patients that died in the first 48 hours of admission
hours_to_death = (hosps['dod'] - hosps['admittime']).dt.total_seconds() / 3600
hosps = hosps[~((hosps['mort'].astype(bool)) & (hours_to_death < min_target_onset))]
print(f"4. Exclude patients who died within {min_target_onset}-hours of admission: N={hosps.shape[0]}")

1. Include only first admissions: N=32513
2. Exclusion by ages: N=25548
3. Include only patients who admitted for at least 24 hours: N=25168
4. Exclude patients who died within 48-hours of admission: N=24825


In [18]:
labs = labs[labs['hadm_id'].isin(hosps['hadm_id'])]
labs = pd.merge(labs,lavbevent_meatdata,on='itemid')
labs = labs[labs['valuenum'].between(labs['min'],labs['max'],  inclusive='both')]

vits = vits[vits['hadm_id'].isin(hosps['hadm_id'])]
vits = pd.merge(vits,vital_meatdata,on='itemid')
vits = vits[vits['valuenum'].between(vits['min'],vits['max'], inclusive='both')]

vits.loc[(vits['feature name'] == 'TempF'),'valuenum'] = (vits[vits['feature name'] == 'TempF']['valuenum']-32)/1.8
vits.loc[vits['feature name'] == 'TempF','feature name'] = 'TempC'

merged = pd.concat([vits, labs])
merged['charttime'] = pd.to_datetime(merged['charttime'], errors='coerce')

pivot = pd.pivot_table(merged, index=['subject_id', 'hadm_id', pd.Grouper(key='charttime', freq=pred_freq)],
                       columns=['feature name'], values='valuenum', aggfunc=['mean', 'max', 'min', 'std'])
pivot.columns = [f'{c[1]}_{c[0]}' for c in pivot.columns.to_flat_index()]

# temp = merged.copy()

merged = pd.merge(hosps, pivot.reset_index(), on=['subject_id', 'hadm_id'])
merged[pivot.columns] = merged.groupby(['subject_id', 'hadm_id'])[pivot.columns].ffill()


/tmp/ipykernel_3463189/9803228.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pivot = pd.pivot_table(merged, index=['subject_id', 'hadm_id', pd.Grouper(key='charttime', freq=pred_freq)],


In [ ]:
# Left join merged with bios_table on (subject_id, hadm_id, charttime)
merged_with_bios = merged[['subject_id', 'hadm_id', 'charttime']].merge(
    bios_table,
    on=['subject_id', 'hadm_id', 'charttime'],
    how='left'
)

onehot_cols = [col for col in merged_with_bios.columns if col.startswith('spec_org_')]
merged_with_bios[onehot_cols] = merged_with_bios[onehot_cols].fillna(0).astype(int)


Original merged shape: (279735, 131)
Bios_table shape: (10727, 241)
Merged with bios shape: (279735, 241)
Number of rows with bios data: 279735
Left join completed and NaN values filled with 0


In [13]:
merged = merged.sort_values(['subject_id', 'hadm_id', 'charttime'])
labs_features_names = set(labs['feature name'])
vits_features_names = set(vits['feature name'])
labs_features = [col for col in merged.columns if col.split('_')[0] in labs_features_names]
vits_features = [col for col in merged.columns if col.split('_')[0] in vits_features_names]

lab_diff_cols = {}
for col in labs_features:
    if col.find("mean") >= 0:
      base = merged.groupby(['subject_id', 'hadm_id'])[col].transform('first')
      lab_diff_cols[f'{col}_diff'] = merged[col] - base

lab_diff_df = pd.DataFrame(lab_diff_cols)

vital_diff_cols = {}
for col in vits_features:
  if col.find("mean") >= 0:
      diff_series = merged.groupby(['subject_id', 'hadm_id'])[col].diff()
      vital_diff_cols[f'{col}_diff'] = diff_series

vital_diff_df = pd.DataFrame(vital_diff_cols)

# Concatenate back to original DataFrame
merged = pd.concat([merged, lab_diff_df, vital_diff_df], axis=1)

In [14]:
merged['charttime'] = pd.to_datetime(merged['charttime'], errors='coerce')

time_to_death = (merged['dod'] - merged['charttime']).dt.total_seconds() / (60 * 60)
merged['target'] = (pd.notnull(time_to_death) & (time_to_death <= pred_window + pred_gap)).astype(int)
merged = merged[time_to_death.isna() | (time_to_death >= pred_gap)]

In [15]:
#min_max_chartime = merged.groupby("subject_id").agg({"charttime": ["min", "max"]})
#min_max_chartime.columns = ['min_charttime', 'max_charttime']
#min_max_chartime.to_csv('data/min_max_chartime.csv')

In [16]:
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

merged_clean = merged.reset_index(drop=True)

np.random.seed(0)

#Split to train & test (all data of a single patient needs to be in the same group)
X = merged_clean
y = merged_clean['target']
groups = merged_clean['subject_id']

gss = GroupShuffleSplit(n_splits=1, train_size=.8, test_size=0.1)
train_index, test_index = next(gss.split(X, y, groups))
val_index = list(set(X.index.to_list()) - (set(train_index.tolist()) | set(test_index.tolist())))

X_train, y_train = X.iloc[train_index], y.iloc[train_index]
X_val, y_val = X.iloc[val_index], y.iloc[val_index]
X_test, y_test = X.iloc[test_index], y.iloc[test_index]

In [17]:
from sklearn.preprocessing import StandardScaler



num_cols = X_train.select_dtypes(include='float').columns
scaler = StandardScaler()

X_train.loc[:, num_cols] = scaler.fit_transform(X_train[num_cols])
X_val.loc[:, num_cols] = scaler.transform(X_val[num_cols])
X_test.loc[:, num_cols] = scaler.transform(X_test[num_cols])

baseline_df = X_train[X_train.charttime.dt.date == X_train.admittime.dt.date].mean(axis=0).fillna(0)
X_train.loc[:, num_cols] = X_train[num_cols].fillna(baseline_df)
X_val.loc[:, num_cols] = X_val[num_cols].fillna(baseline_df)
X_test.loc[:, num_cols] = X_test[num_cols].fillna(baseline_df)

In [18]:
to_drop = ['hadm_id','icustay_id','intime','outtime','admittime', 'dischtime', 'dod','dob', 'mort', 'los_hosp_hr', 'charttime','adm_to_death']

to_keep = ~X_train.columns.isin(to_drop)
to_keep = X_train.columns[to_keep]
X_train = X_train[to_keep]
X_test = X_test[to_keep]
X_val = X_val[to_keep]

In [19]:
from sklearn.cluster import KMeans
import numpy as np

def cluster_and_select_subjects(X_train, num_clusters=10, random_state=42):
    """
    Calculate the first row of each subject_id in X_train, cluster it to num_clusters 
    and choose one subject_id from each cluster.
    
    Parameters:
    X_train: DataFrame with subject_id column
    num_clusters: int, number of clusters to create
    random_state: int, for reproducibility
    
    Returns:
    list: selected subject_ids, one from each cluster
    """
    first_rows = X_train.groupby('subject_id').first().reset_index()
    
    features_for_clustering = first_rows.drop('subject_id', axis=1)
    

    kmeans = KMeans(n_clusters=num_clusters, random_state=random_state, n_init=10)
    cluster_labels = kmeans.fit_predict(features_for_clustering)
    

    first_rows['cluster'] = cluster_labels
    

    np.random.seed(random_state)
    selected_subjects = []
    
    for cluster_id in range(num_clusters):
        cluster_subjects = first_rows[first_rows['cluster'] == cluster_id]['subject_id'].values
        if len(cluster_subjects) > 0:
            selected_subject = np.random.choice(cluster_subjects)
            selected_subjects.append(selected_subject)
    
    return selected_subjects

selected_subjects = cluster_and_select_subjects(X_train, num_clusters=150, random_state=42)

In [20]:
# Create X_core with rows where subject_id is in selected_subjects
X_core = X_train[X_train['subject_id'].isin(selected_subjects)]

# Update X_train to exclude the selected subjects
X_train = X_train[~X_train['subject_id'].isin(selected_subjects)]

In [21]:
def generate_series_data(df, group_col="subject_id", maxlen=18):
  grouped = df.groupby(group_col)
  subject_sequences = [group.values[:, 1:] for _, group in grouped]
  padded_tensor = pad_sequences(subject_sequences, padding='post', dtype='float32')
  sequence_lengths = [len(seq) for seq in subject_sequences]
  padding_mask = np.zeros((len(sequence_lengths), maxlen), dtype=np.float32)
  for i, length in enumerate(sequence_lengths):
      padding_mask[i, :length] = 1.0
  labels = padded_tensor[:,:,-1]
  padded_tensor = padded_tensor[:,:,:-1]
  padded_tensor = torch.tensor(padded_tensor, dtype=torch.float32)
  labels = torch.tensor(labels, dtype=torch.float32)
  padding_mask = torch.tensor(padding_mask, dtype=torch.float32)
  return padded_tensor, labels, padding_mask

In [22]:
#import NoteEmbedder
#import imp
#os.environ["TRANSFORMERS_CACHE"] = "/home/bnet/ronsheinin/MLHC/MLHC/cache"
#imp.reload(NoteEmbedder)
#import os
#notes = pd.read_csv('data/notes.csv')
#note_embeddings = NoteEmbedder.run_embeeding(notes)

## Model

In [24]:
# def build_knn_graph2(batch, core, padding_mask_batch, padding_mask_core, k=5):
#         """
#         Build a KNN graph from batch and core tensors.
        
#         Args:
#             batch: 3D tensor (batch_size, seq_len, features)
#             core: 3D tensor (core_size, seq_len, features)
#             padding_mask_batch: 2D tensor (batch_size, seq_len) indicating valid time points
#             padding_mask_core: 2D tensor (core_size, seq_len) indicating valid time points
#             k: number of nearest neighbors for patient connections
        
#         Returns:
#             edge_index: tensor of shape (2, num_edges) representing graph edges
#         """
#         batch_size, seq_len, _ = batch.shape
#         core_size = core.shape[0]
#         total_patients = batch_size + core_size
#         batch_size = batch.shape[0]

#         all_patients = torch.cat([batch, core], dim=0)
#         all_padding_mask = torch.cat([padding_mask_batch, padding_mask_core], dim=0)
        
#         edges = []
        
#         for patient_idx in range(total_patients):
#             for t in range(seq_len - 1):
            
#                 if all_padding_mask[patient_idx, t] > 0 and all_padding_mask[patient_idx, t + 1] > 0:
#                     node_curr = patient_idx * seq_len + t
#                     node_next = patient_idx * seq_len + t + 1
   
#                     edges.append([node_curr, node_next])
#                     edges.append([node_next, node_curr])
        
        
#         for t in range(seq_len):
#             valid_patients = all_padding_mask[:, t] > 0
#             valid_indices = torch.where(valid_patients)[0] 
            

#             valid_patients_to = all_padding_mask[batch_size:, t] > 0
#             valid_indices_to = torch.where(valid_patients_to)[0] + batch_size


#             if len(valid_indices) > 1:
#                 features_t = all_patients[valid_indices, t, :]
#                 features_t_to = all_patients[valid_indices_to, t, :]

#                 distances = torch.cdist(features_t, features_t_to, p=2)
                
#                 for i, patient_idx in enumerate(valid_indices):
#                     num_neighbors = min(k, len(valid_indices_to))
#                     _, nearest_indices = torch.topk(distances[i], num_neighbors, largest=False)
                    
#                     for j in nearest_indices:
#                         neighbor_idx = valid_indices_to[j]
#                         node_curr = patient_idx * seq_len + t
#                         node_neighbor = neighbor_idx * seq_len + t
#                         edges.append([node_neighbor, node_curr])

        
#         if edges:
#             edge_index = torch.tensor(edges, dtype=torch.long).t()
#         else:
#             edge_index = torch.empty((2, 0), dtype=torch.long)
        
#         return edge_index

In [33]:
import torch
from torch.utils.data import Dataset


class PatientDataset(Dataset):
    def __init__(self, X, y, core, padding_mask, padding_mask_core, k=5):
        self.core = core
        self.X = X
        self.y = y
        self.padding_mask = padding_mask
        self.padding_mask_core = padding_mask_core
        self.k = k
        self.node_to_neighbors = self.build_knn_graph(X, core, padding_mask, padding_mask_core, k=k)


    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.padding_mask[idx], idx
    
    # def cal_graphs(self):
    #     if self.edge_list is not None:
    #         return self.edge_list
        
    #     edge_list = []
    #     for patient_idx in range(len(self.X)):
    #         edges = self.build_knn_graph(self.X[patient_idx:patient_idx+1], self.core, 
    #                                             self.padding_mask[patient_idx:patient_idx+1], self.padding_mask_core, k=self.k)
    #         edge_list.append(edges)
    #     self.edge_list = torch.cat(edge_list, dim=1).T

    def get_edge_index(self, batch, padding_mask_batch, batch_indices):
        """
        Returns a tensor representing the edge index of the graph.
        
        Returns:
            edge_index: tensor of shape (2, num_edges) representing graph edges
        """
        batch_size, seq_len, _ = batch.shape
        core_size = self.core.shape[0]
        total_patients = batch_size + core_size

        batch_size = batch.shape[0]
        # all_patients = torch.cat([batch, self.core], dim=0)
        all_padding_mask = torch.cat([padding_mask_batch, self.padding_mask_core.to(padding_mask_batch.device)], dim=0)

        edges = []
        
        for patient_idx in range(total_patients):
            for t in range(seq_len - 1):
                if all_padding_mask[patient_idx, t] > 0 and all_padding_mask[patient_idx, t + 1] > 0:
                    node_curr = patient_idx * seq_len + t
                    node_next = patient_idx * seq_len + t + 1
   
                    edges.append([node_curr, node_next])
                    edges.append([node_next, node_curr])
        
        edges = [torch.tensor(edges).t()]

        neighbors_list = []
        for idx in batch_indices:
            neighbors_list.extend(self.node_to_neighbors[idx.item() * seq_len: (idx.item() + 1) * seq_len])
            # for i in range(seq_len):
            #     neighbors_list.append(list(self.node_to_neighbors[idx.item() * seq_len + i]))
        
        # for i in range(self.X.shape[0] * seq_len, self.X.shape[0] * seq_len + core_size * seq_len):
        #     neighbors_list.append(list(self.node_to_neighbors[i]))
        neighbors_list.extend(self.node_to_neighbors[self.X.shape[0] * seq_len:])
    
        for t in range(seq_len):
            valid_patients = all_padding_mask[:, t] > 0
            valid_indices = torch.where(valid_patients)[0] 

            if len(valid_indices) > 1:
                for i, patient_idx in enumerate(valid_indices):
                    node_curr = patient_idx * seq_len + t
                    to_append = torch.tensor(neighbors_list[node_curr], dtype=torch.long) - (self.X.shape[0] - batch_size) * seq_len  # Adjust index for core patients
                    edges.append(torch.stack([to_append, torch.full_like(to_append, node_curr)], dim=0))

        
        if edges:
            edge_index = torch.cat(edges, dim=1).to(torch.long)
        else:
            edge_index = torch.empty((2, 0), dtype=torch.long)
        
        return edge_index


        
    @staticmethod
    def build_knn_graph(batch, core, padding_mask_batch, padding_mask_core, k=5):
        """
        Build a KNN graph from batch and core tensors.
        
        Args:
            batch: 3D tensor (batch_size, seq_len, features)
            core: 3D tensor (core_size, seq_len, features)
            padding_mask_batch: 2D tensor (batch_size, seq_len) indicating valid time points
            padding_mask_core: 2D tensor (core_size, seq_len) indicating valid time points
            k: number of nearest neighbors for patient connections
        
        Returns:
            edge_index: tensor of shape (2, num_edges) representing graph edges
        """

        batch_size, seq_len, _ = batch.shape
        core_size = core.shape[0]
        total_patients = batch_size + core_size
        batch_size = batch.shape[0]
        all_patients = torch.cat([batch, core], dim=0)
        all_padding_mask = torch.cat([padding_mask_batch, padding_mask_core], dim=0)
        
        node_to_neighbors = {i: set() for i in range(total_patients * seq_len)}

        edges = []
           
        for t in range(seq_len):
            valid_patients = all_padding_mask[:, t] > 0
            valid_indices = torch.where(valid_patients)[0] 

            core_patients = all_padding_mask[batch_size:, t] > 0
            core_indices = torch.where(core_patients)[0] + batch_size

            if len(valid_indices) > 1:
                features_t = all_patients[valid_indices, t, :]
                features_t_to = all_patients[core_indices, t, :]

                distances = torch.cdist(features_t, features_t_to, p=2)
                
                for i, patient_idx in enumerate(valid_indices):
                    num_neighbors = min(k, len(core_indices))
                    _, nearest_indices = torch.topk(distances[i], num_neighbors, largest=False)
                    
                    for j in nearest_indices:
                        neighbor_idx = core_indices[j]
                        node_curr = patient_idx * seq_len + t
                        node_neighbor = neighbor_idx * seq_len + t

                        edges.append([node_neighbor, node_curr])
                        
                        node_to_neighbors[node_curr.item()].add(node_neighbor.item())
        
        result = [list(node_to_neighbors[i]) for i in range(total_patients * seq_len)]
        return result

# dataset = PatientDataset(core=padd_tensor_core, X=padded_tensor_train, y=labels_train, padding_mask=padding_mask_train, padding_mask_core=padding_mask_core, k=5)
# dataset.cal_graphs()

In [34]:
from sklearn.metrics import average_precision_score, roc_auc_score


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#DEVICE = torch.device('cpu')  # For testing purposes, use CPU


class GraphGRUMortalityModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, n1_gat_layers, n2_gru_layers, X_core, core_padding_mask,
                 num_heads=4, dropout=0.1, seq_len=18, k=5, gnn_flag=True):
        """
        Mortality prediction model with Graph Attention + GRU layers
        
        Args:
            input_dim: Input feature dimension
            hidden_dim: Hidden dimension for GAT and GRU layers
            n1_gat_layers: Number of Graph Attention layers
            n2_gru_layers: Number of GRU layers
            X_core_dim: Core set dimension (number of core patients)
            num_heads: Number of attention heads for GAT
            dropout: Dropout rate
            seq_len: Sequence length
        """
        super(GraphGRUMortalityModel, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n1_gat_layers = n1_gat_layers
        self.n2_gru_layers = n2_gru_layers
        self.X_core = X_core.to(DEVICE)
        self.core_padding_mask = core_padding_mask.to(DEVICE)
        self.seq_len = seq_len
        self.num_heads = num_heads
        self.gnn_flag = gnn_flag
        
        self.gat_layers = nn.ModuleList().to(DEVICE)
        
        
        self.gat_layers.append(
            GATv2Conv(input_dim, hidden_dim // num_heads, heads=num_heads, dropout=dropout, concat=True)
        )
        
        for _ in range(n1_gat_layers - 1):
            self.gat_layers.append(
                GATv2Conv(hidden_dim, hidden_dim // num_heads, heads=num_heads, dropout=dropout, concat=True)
            )
        
        if self.gnn_flag:
            self.gru = nn.GRU(hidden_dim, hidden_dim, n2_gru_layers, batch_first=True, dropout=dropout)
        else:
            self.gru = nn.GRU(input_dim, hidden_dim, n2_gru_layers, batch_first=True, dropout=dropout)
        
        self.classifier = nn.Linear(hidden_dim, 1).to(DEVICE)
        self.dropout = nn.Dropout(dropout).to(DEVICE)
        self.k = k

        self.best_model = None

        # initialize weights
        for layer in self.gat_layers:
            if isinstance(layer, GATv2Conv):
                nn.init.xavier_uniform_(layer.lin_l.weight)
                nn.init.xavier_uniform_(layer.lin_r.weight) 
        nn.init.xavier_uniform_(self.classifier.weight) 
        nn.init.constant_(self.classifier.bias, 0.0)
        nn.init.xavier_uniform_(self.gru.weight_ih_l0)
        nn.init.xavier_uniform_(self.gru.weight_hh_l0)
        nn.init.constant_(self.gru.bias_ih_l0, 0.0)
        nn.init.constant_(self.gru.bias_hh_l0, 0.0)
        
  
        
    def forward(self, x ,padding_mask, edge_index):
        """
        Forward pass
        
        Args:
            core: Core patients tensor (X_core_dim, seq_len, input_dim)
            x: Batch patients tensor (batch_size, seq_len, input_dim)
            y: Target labels (batch_size, seq_len)
            padding_mask: Padding mask (batch_size, seq_len)
            edge_index: Graph edge indices (2, num_edges)
        
        Returns:
            predictions: Mortality predictions (batch_size, seq_len, 1)
        """
        batch_size = x.size(0)
        
        if self.gnn_flag:
            all_patients = torch.cat([x, self.X_core], dim=0)  # (batch_size + X_core_dim, seq_len, input_dim)
            total_patients = batch_size + self.X_core.shape[0]
            
            # Reshape for graph processing: (total_patients * seq_len, input_dim)
            graph_input = all_patients.view(total_patients * self.seq_len, -1)
            # Apply GAT layers
            for gat_layer in self.gat_layers:
                graph_input = F.relu(gat_layer(graph_input, edge_index))
            
            # Reshape back to sequence format: (total_patients, seq_len, hidden_dim)
            graph_output = graph_input.view(total_patients, self.seq_len, -1)

            # Extract only batch patients (exclude core)
        
            batch_output = graph_output[:batch_size]  # (batch_size, seq_len, hidden_dim)
        
        else:
            batch_output = x
        # Apply GRU layers
        # Pack sequences for efficient processing
        # lengths from mask (True = pad) → count valid steps
        lengths = (padding_mask.to(bool)).sum(dim=1)                       # (batch,)
        lengths = lengths.clamp(min=1).cpu()
        #packed_input = pack_padded_sequence(batch_output, lengths, batch_first=True, enforce_sorted=False)
        #gru_output, _ = self.gru(packed_input)
        # Unpack sequences
        #gru_output, _ = pad_packed_sequence(gru_output, batch_first=True, total_length=self.seq_len)


        gru_output, _ = self.gru(batch_output)
        # Apply classifier
        predictions = self.classifier(gru_output)  # (batch_size, seq_len, 1)
        
        return predictions.squeeze(-1)  # (batch_size, seq_len)
    
    
    def masked_bce_loss(self, logits, targets, mask, pos_weight=None):
        T = min(logits.shape[1], targets.shape[1], mask.shape[1])
        logits, targets, mask = logits[:, :T], targets[:, :T], mask[:, :T]
        loss = nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)(logits, targets)
        return (loss * mask).sum() / mask.sum()
    

    def train_all(self, dataloaders, datasets, epochs: int = 10, learning_rate: float = 1e-3, pos_lambda : float = 1):
        self.train()
        optim = torch.optim.Adam(self.parameters(), lr=learning_rate)
        best_validation_accuracy = - float('inf')
        targets = datasets['train'].y
        pos_weight = (targets == 0).sum() / (targets == 1).sum()  # Adjust pos_weight as needed
        print(f'Pos weight: {pos_weight:.4f}')


        for epoch in range(epochs):
            print(f'Starting epoch {epoch + 1}/{epochs}')
            total = 0
            for x, y, padding_mask, idx in tqdm(dataloaders['train']):
                optim.zero_grad()

                x, padding_mask, y = x.to(DEVICE), padding_mask.to(DEVICE), y.to(DEVICE)
                edge_index = datasets['train'].get_edge_index(x, padding_mask, idx).to(DEVICE)
                predictions = self.forward(x, padding_mask, edge_index)
                loss = self.masked_bce_loss(
                    predictions, y, padding_mask, pos_weight=pos_weight * pos_lambda
                )
                #loss = F.binary_cross_entropy_with_logits(predictions.view(-1), y.view(-1))
                loss.backward()
                optim.step()
                
                total += loss.item()
            avg_loss = total / len(dataloaders['train'])
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}')
            validation_accuracy = self.validate(dataloaders['val'], datasets['val'])
            print(f'Validation Accuracy: {validation_accuracy[0]:.4f} | AUC: {validation_accuracy[1]:.4f} | AP: {validation_accuracy[2]:.4f}')
            if validation_accuracy[2] > best_validation_accuracy:
                best_validation_accuracy = validation_accuracy[2]
                self.best_model = self.state_dict()
                print("Best model updated")
        self.load_state_dict(self.best_model)

    def validate(self, dataloader, dataset):
        self.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            all_true_labels = []
            all_predicted_labels = []
            for x, y, padding_mask, idx in tqdm(dataloader):
                x, padding_mask, y = x.to(DEVICE), padding_mask.to(DEVICE), y.to(DEVICE)
                # edge_index = dataset.build_knn_graph(x, self.X_core, padding_mask, self.core_padding_mask, k=self.k).to(DEVICE)
                edge_index = dataset.get_edge_index(x, padding_mask, idx).to(DEVICE)
                predictions = self.forward(x, padding_mask, edge_index)
                all_true_labels.extend(y.cpu().numpy().flatten())
                all_predicted_labels.extend(torch.sigmoid(predictions).cpu().numpy().flatten())
                predicted_labels = (torch.sigmoid(predictions) > 0.5).float()
                correct += (predicted_labels == y).sum().item()
                total += y.numel()
        accuracy = correct / total if total > 0 else 0
        self.train()
        return accuracy, roc_auc_score(all_true_labels, all_predicted_labels), average_precision_score(all_true_labels, all_predicted_labels)


    


In [35]:
batch_size = 256
hidden_dim = 64
k = 11

padded_tensor_train, labels_train, padding_mask_train = generate_series_data(X_train, group_col="subject_id", maxlen=18)
padded_tensor_val, labels_val, padding_mask_val = generate_series_data(X_val, group_col="subject_id", maxlen=18)
padded_tensor_test, labels_test, padding_mask_test = generate_series_data(X_test, group_col="subject_id", maxlen=18)
padd_tensor_core, labels_core, padding_mask_core = generate_series_data(X_core, group_col="subject_id", maxlen=18)

datasets = {x: PatientDataset(d, y, core=padd_tensor_core, padding_mask=m, padding_mask_core=padding_mask_core, k=k) for x, d, y, m in
        zip(['train', 'val', 'test'], [padded_tensor_train, padded_tensor_val, padded_tensor_test], 
            [labels_train, labels_val, labels_test], 
            [padding_mask_train, padding_mask_val, padding_mask_test])}
dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val', 'test']}


In [36]:
model = GraphGRUMortalityModel(input_dim=padded_tensor_train.shape[2], hidden_dim=hidden_dim, n1_gat_layers=2, n2_gru_layers=2, X_core=padd_tensor_core, 
                               core_padding_mask=padding_mask_core, num_heads=4, dropout=0.1, seq_len=padded_tensor_train.shape[1], k=k, gnn_flag=True).to(DEVICE)

model.train_all(dataloaders, datasets, epochs=15, learning_rate=1e-3)
print("Training completed. Validating on test set...")

acc, auc, pr =  model.validate(dataloaders['test'], datasets['test'])
print(f'Test Accuracy in BaseLine model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

Pos weight: 20.2490
Starting epoch 1/15


  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [00:50<00:00,  1.54it/s]


Epoch 1/15, Loss: 1.5944


100%|██████████| 10/10 [00:06<00:00,  1.62it/s]


...

Epoch 15/15, Loss: 1.1669


100%|██████████| 10/10 [00:06<00:00,  1.63it/s]


Validation Accuracy: 0.5813 | AUC: 0.8371 | AP: 0.2379
Training completed. Validating on test set...


100%|██████████| 10/10 [00:06<00:00,  1.62it/s]

Test Accuracy in BaseLine model: 0.5691, AUC: 0.8350, AP: 0.2529


In [ ]:

acc, auc, pr =  model.validate(dataloaders['test'], datasets['test'])
print(f'Test Accuracy in BaseLine model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

Test Accuracy in BaseLine model: 0.7342, AUC: 0.8546, AP: 0.2779


In [ ]:
import optuna
import gc
import joblib
def objective_mlp(trial):
    hidden_dim = trial.suggest_int('hidden_dim', 32, 128, step=16)
    n1_gat_layers = trial.suggest_int('n1_gat_layers', 1, 5)
    n2_gru_layers = trial.suggest_int('n2_gru_layers', 1, 5)
    dropout = trial.suggest_float('dropout', 0.1, 0.5, step=0.1)
    num_heads = trial.suggest_int('num_heads', 2, 8, step=2)
    k = trial.suggest_int('k', 5, 15, step=2)

    model = GraphGRUMortalityModel(input_dim=padded_tensor_train.shape[2], hidden_dim=hidden_dim, n1_gat_layers=n1_gat_layers,
                                   n2_gru_layers=n2_gru_layers, X_core=padd_tensor_core, core_padding_mask=padding_mask_core,
                                   num_heads=num_heads, dropout=dropout, seq_len=padded_tensor_train.shape[1], k=k, gnn_falg=True).to(DEVICE)

    model.train_all(dataloaders, datasets, epochs=10, learning_rate=1e-3)
    
    acc, auc, pr = model.validate(dataloaders['test'], datasets['test'])
    
    del model 
    torch.cuda.empty_cache() 
    gc.collect()
    return pr

batch_size = 256

study = optuna.create_study(direction='maximize')
study.optimize(objective_mlp, n_trials=20)  
print("Best trial:")
print(study.best_trial)
print("Best hyperparameters:", study.best_trial.params) 
joblib.dump(study, 'optuna_study.pkl')


In [27]:
print(study.best_trial)
print("Best hyperparameters:", study.best_trial.params) 

FrozenTrial(number=1, state=TrialState.COMPLETE, values=[0.3055693844824112], datetime_start=datetime.datetime(2025, 8, 17, 19, 18, 21, 898015), datetime_complete=datetime.datetime(2025, 8, 17, 19, 53, 35, 836441), params={'hidden_dim': 48, 'n1_gat_layers': 3, 'n2_gru_layers': 2, 'dropout': 0.1, 'num_heads': 8, 'k': 5}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden_dim': IntDistribution(high=128, log=False, low=32, step=16), 'n1_gat_layers': IntDistribution(high=5, log=False, low=1, step=1), 'n2_gru_layers': IntDistribution(high=5, log=False, low=1, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.1, step=0.1), 'num_heads': IntDistribution(high=8, log=False, low=2, step=2), 'k': IntDistribution(high=15, log=False, low=5, step=2)}, trial_id=1, value=None)
Best hyperparameters: {'hidden_dim': 48, 'n1_gat_layers': 3, 'n2_gru_layers': 2, 'dropout': 0.1, 'num_heads': 8, 'k': 5}


In [92]:
hidden_dim = study.best_trial.params['hidden_dim']
n1_gat_layers = study.best_trial.params['n1_gat_layers']
n2_gru_layers = study.best_trial.params['n2_gru_layers']
dropout = study.best_trial.params['dropout']
num_heads = study.best_trial.params['num_heads']
k = study.best_trial.params['k']

model = GraphGRUMortalityModel(input_dim=padded_tensor_train.shape[2], hidden_dim=hidden_dim, n1_gat_layers=n1_gat_layers,
                            n2_gru_layers=n2_gru_layers, X_core=padd_tensor_core, core_padding_mask=padding_mask_core,
                            num_heads=num_heads, dropout=dropout, seq_len=padded_tensor_train.shape[1], k=k, gnn_falg=True).to(DEVICE)


model.train_all(dataloaders, datasets, epochs=15, learning_rate=1e-3)
print("Training completed. Validating on test set...")

acc, auc, pr =  model.validate(dataloaders['test'], datasets['test'])
print(f'Test Accuracy in GNN model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

Starting epoch 1/15


100%|██████████| 78/78 [02:54<00:00,  2.24s/it]


Epoch 1/15, Loss: 0.2867
Validation Accuracy: 0.9511 | AUC: 0.4619 | AP: 0.0443
Best model updated
Starting epoch 2/15


100%|██████████| 78/78 [02:54<00:00,  2.24s/it]


Epoch 2/15, Loss: 0.2454
Validation Accuracy: 0.9509 | AUC: 0.6103 | AP: 0.1005
Best model updated
Starting epoch 3/15


...

Epoch 13/15, Loss: 0.2093
Validation Accuracy: 0.9496 | AUC: 0.7764 | AP: 0.2037
Starting epoch 14/15


100%|██████████| 78/78 [02:53<00:00,  2.23s/it]


Epoch 14/15, Loss: 0.2096
Validation Accuracy: 0.9503 | AUC: 0.7686 | AP: 0.1972
Starting epoch 15/15


100%|██████████| 78/78 [02:51<00:00,  2.20s/it]


Epoch 15/15, Loss: 0.2072
Validation Accuracy: 0.9416 | AUC: 0.7864 | AP: 0.1931
Training completed. Validating on test set...
Test Accuracy in GNN model: 0.9417, AUC: 0.8056, AP: 0.1985


In [ ]:
model = GraphGRUMortalityModel(input_dim=padded_tensor_train.shape[2], hidden_dim=hidden_dim, n1_gat_layers=2, n2_gru_layers=2, X_core=padd_tensor_core, 
                               core_padding_mask=padding_mask_core, num_heads=4, dropout=0.1, seq_len=padded_tensor_train.shape[1], k=k, gnn_falg=False).to(DEVICE)

model.train_all(dataloaders, datasets, epochs=15, learning_rate=1e-3)
print("Training completed. Validating on test set...")

acc, auc, pr =  model.validate(dataloaders['test'], datasets['test'])
print(f'Test Accuracy in GNN model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')